## Chapter 16: Testing Your Application (Industry Standard)

Building a feature‑rich application is only half the battle. To ensure it remains reliable as it grows, you need a solid testing strategy. Automated tests give you confidence to refactor, catch regressions early, and document expected behavior. In the .NET ecosystem, a rich testing culture has emerged, with tools like xUnit, Moq, and the built‑in `WebApplicationFactory` making it straightforward to test every layer of your ASP.NET Core application. In this chapter, you’ll learn how to write unit tests for isolated components, mock dependencies to control test environments, perform integration tests that exercise your full stack, and specifically test Web API controllers. By the end, you’ll be equipped to build a comprehensive test suite that follows industry best practices.

### 16.1 Unit Testing with xUnit or NUnit

**Unit testing** focuses on testing the smallest testable parts of an application in isolation (e.g., a single method or class). The goal is to verify that each unit works correctly independently of external dependencies like databases, file systems, or networks.

#### Choosing a Testing Framework

The two most popular unit testing frameworks in .NET are:

- **xUnit.net**: The modern, extensible framework used by Microsoft for the .NET runtime and ASP.NET Core. It’s the default choice for new projects.
- **NUnit**: A mature framework with a rich set of assertions and a long history. Still widely used, especially in legacy projects.

In this chapter, we’ll use **xUnit**, but the concepts apply to any framework.

#### Setting Up a Test Project

Create a new xUnit test project in your solution:

```bash
dotnet new xunit -n MyApp.Tests
```

Add a reference to the project you want to test (e.g., your main web application project):

```bash
dotnet add MyApp.Tests reference ../MyApp/MyApp.csproj
```

You may also need to install NuGet packages for mocking, assertions, etc., depending on your needs.

#### Your First Unit Test

Let’s say you have a simple service that calculates discounts:

```csharp
public class DiscountService
{
    public decimal CalculateDiscount(decimal price, bool isPreferredCustomer)
    {
        if (price <= 0) throw new ArgumentException("Price must be positive.");
        if (isPreferredCustomer) return price * 0.10m; // 10% discount
        return price * 0.05m; // 5% discount
    }
}
```

A unit test for this service might look like:

```csharp
using Xunit;

namespace MyApp.Tests
{
    public class DiscountServiceTests
    {
        [Fact]
        public void CalculateDiscount_PreferredCustomer_Returns10Percent()
        {
            // Arrange
            var service = new DiscountService();
            var price = 100m;

            // Act
            var result = service.CalculateDiscount(price, true);

            // Assert
            Assert.Equal(10m, result);
        }

        [Fact]
        public void CalculateDiscount_RegularCustomer_Returns5Percent()
        {
            // Arrange
            var service = new DiscountService();
            var price = 100m;

            // Act
            var result = service.CalculateDiscount(price, false);

            // Assert
            Assert.Equal(5m, result);
        }

        [Fact]
        public void CalculateDiscount_NegativePrice_ThrowsArgumentException()
        {
            // Arrange
            var service = new DiscountService();
            var price = -10m;

            // Act & Assert
            Assert.Throws<ArgumentException>(() => service.CalculateDiscount(price, true));
        }
    }
}
```

- `[Fact]` marks a test method that takes no parameters.
- `Assert` class provides various assertion methods (e.g., `Equal`, `True`, `Throws`).
- Tests follow the **Arrange‑Act‑Assert** pattern: set up the test (Arrange), execute the code under test (Act), and verify the outcome (Assert).

#### Using `[Theory]` and `[InlineData]`

For tests that differ only by input parameters, you can use `[Theory]` with `[InlineData]` to create data‑driven tests.

```csharp
[Theory]
[InlineData(100, true, 10)]
[InlineData(100, false, 5)]
[InlineData(200, true, 20)]
[InlineData(200, false, 10)]
public void CalculateDiscount_ValidInputs_ReturnsExpectedDiscount(decimal price, bool preferred, decimal expected)
{
    var service = new DiscountService();
    var result = service.CalculateDiscount(price, preferred);
    Assert.Equal(expected, result);
}
```

This runs four separate tests, making it easy to add new cases.

### 16.2 Mocking Dependencies with Moq (or NSubstitute)

Most classes have dependencies. Unit testing requires isolating the class under test from its dependencies, so you can control their behavior and verify interactions. **Mocking frameworks** like **Moq** (or NSubstitute) allow you to create fake implementations of interfaces or abstract classes.

#### Installing Moq

```bash
dotnet add package Moq
```

#### Example: Testing a Service That Uses a Repository

Assume you have a `ProductService` that depends on `IProductRepository`:

```csharp
public class ProductService
{
    private readonly IProductRepository _productRepository;

    public ProductService(IProductRepository productRepository)
    {
        _productRepository = productRepository;
    }

    public async Task<Product> GetProductAsync(int id)
    {
        var product = await _productRepository.GetByIdAsync(id);
        if (product == null)
            throw new KeyNotFoundException($"Product {id} not found.");
        return product;
    }
}
```

To test `ProductService.GetProductAsync` in isolation, you create a mock `IProductRepository`:

```csharp
using Moq;
using Xunit;

public class ProductServiceTests
{
    [Fact]
    public async Task GetProductAsync_ExistingId_ReturnsProduct()
    {
        // Arrange
        var mockRepo = new Mock<IProductRepository>();
        var expectedProduct = new Product { Id = 1, Name = "Test" };
        mockRepo.Setup(repo => repo.GetByIdAsync(1))
                .ReturnsAsync(expectedProduct);

        var service = new ProductService(mockRepo.Object);

        // Act
        var result = await service.GetProductAsync(1);

        // Assert
        Assert.Equal(expectedProduct, result);
        mockRepo.Verify(repo => repo.GetByIdAsync(1), Times.Once);
    }

    [Fact]
    public async Task GetProductAsync_NonExistingId_ThrowsKeyNotFoundException()
    {
        // Arrange
        var mockRepo = new Mock<IProductRepository>();
        mockRepo.Setup(repo => repo.GetByIdAsync(999))
                .ReturnsAsync((Product)null);

        var service = new ProductService(mockRepo.Object);

        // Act & Assert
        await Assert.ThrowsAsync<KeyNotFoundException>(() => service.GetProductAsync(999));
    }
}
```

**Key Moq concepts:**

- `Mock<T>` creates a mock object.
- `Setup` defines behavior for a method when called with specific arguments. Use `It.IsAny<int>()` to match any integer.
- `ReturnsAsync` specifies an async return value.
- `Verify` checks that a method was called with expected arguments and the specified number of times.

Moq also supports `Callback`, `Throws`, and property mocking.

#### Alternatives: NSubstitute

NSubstitute offers a more fluent, Arrange‑Act‑Assert syntax. The same test with NSubstitute:

```csharp
var repo = Substitute.For<IProductRepository>();
repo.GetByIdAsync(1).Returns(expectedProduct);

var service = new ProductService(repo);
var result = await service.GetProductAsync(1);

Assert.Equal(expectedProduct, result);
repo.Received(1).GetByIdAsync(1);
```

Choose the framework you’re most comfortable with; both are widely used.

### 16.3 Integration Testing with `WebApplicationFactory<T>`

**Integration tests** verify that multiple components work together correctly. In ASP.NET Core, you can spin up an in‑memory test server that hosts your application, allowing you to send real HTTP requests and examine responses. The `WebApplicationFactory<T>` class, part of the `Microsoft.AspNetCore.Mvc.Testing` package, makes this easy.

#### Setting Up Integration Tests

1. Install the required package in your test project:

```bash
dotnet add package Microsoft.AspNetCore.Mvc.Testing
```

2. Create a test class that uses `WebApplicationFactory<T>`, where `T` is your application’s `Program` class (or startup class). For .NET 6+ with top‑level statements, you need to make `Program` accessible. The easiest way is to add a line in your `.csproj` file:

```xml
<ItemGroup>
  <InternalsVisibleTo Include="MyApp.Tests" />
</ItemGroup>
```

Or, you can create a partial `Program` class. We’ll assume the visibility is set.

#### Basic Integration Test

```csharp
using Microsoft.AspNetCore.Mvc.Testing;
using System.Net;
using System.Net.Http.Json;
using Xunit;

public class ProductsApiIntegrationTests : IClassFixture<WebApplicationFactory<Program>>
{
    private readonly HttpClient _client;

    public ProductsApiIntegrationTests(WebApplicationFactory<Program> factory)
    {
        _client = factory.CreateClient();
    }

    [Fact]
    public async Task GetProducts_ReturnsSuccessStatusCode()
    {
        // Act
        var response = await _client.GetAsync("/api/products");

        // Assert
        response.EnsureSuccessStatusCode(); // Status Code 200-299
        Assert.Equal(HttpStatusCode.OK, response.StatusCode);
    }

    [Fact]
    public async Task GetProduct_WithExistingId_ReturnsProduct()
    {
        // Arrange - assuming there's a seeded product with id=1
        var productId = 1;

        // Act
        var product = await _client.GetFromJsonAsync<ProductDto>($"/api/products/{productId}");

        // Assert
        Assert.NotNull(product);
        Assert.Equal(productId, product.Id);
    }
}
```

- `IClassFixture<T>` tells xUnit to create a single instance of the factory and share it across tests (to reduce overhead).
- `CreateClient()` returns an `HttpClient` configured to talk to the in‑memory server.

#### Customizing the Factory

You often need to override application configuration for tests (e.g., use an in‑memory database, mock external services). You can create a custom factory:

```csharp
public class CustomWebApplicationFactory : WebApplicationFactory<Program>
{
    protected override void ConfigureWebHost(IWebHostBuilder builder)
    {
        builder.ConfigureServices(services =>
        {
            // Remove the real DbContext registration
            var descriptor = services.SingleOrDefault(
                d => d.ServiceType == typeof(DbContextOptions<AppDbContext>));
            if (descriptor != null) services.Remove(descriptor);

            // Add an in‑memory database for testing
            services.AddDbContext<AppDbContext>(options =>
            {
                options.UseInMemoryDatabase("TestDb");
            });

            // Optionally, replace other services with mocks
            var emailServiceDescriptor = services.SingleOrDefault(
                d => d.ServiceType == typeof(IEmailService));
            if (emailServiceDescriptor != null) services.Remove(emailServiceDescriptor);
            services.AddScoped<IEmailService, MockEmailService>();
        });

        // Optionally, set environment to "Testing"
        builder.UseEnvironment("Testing");
    }
}
```

Then use this factory in your tests.

#### Seeding Data for Tests

You can seed the in‑memory database in the factory configuration:

```csharp
var sp = services.BuildServiceProvider();
using var scope = sp.CreateScope();
var db = scope.ServiceProvider.GetRequiredService<AppDbContext>();
db.Database.EnsureCreated();
db.Products.Add(new Product { Id = 1, Name = "Test Product", Price = 9.99m });
db.SaveChanges();
```

Be careful with parallel test execution; use unique database names or ensure isolation.

### 16.4 Testing Web APIs and Controllers

You can test controllers in isolation (unit tests) or as part of integration tests. Both have their place.

#### Unit Testing Controllers (with mocked dependencies)

For fine‑grained testing of controller logic, unit test the controller action by mocking its dependencies.

```csharp
public class ProductsControllerTests
{
    [Fact]
    public async Task GetProduct_ExistingId_ReturnsOkWithProduct()
    {
        // Arrange
        var mockService = new Mock<IProductService>();
        var product = new ProductDto { Id = 1, Name = "Test" };
        mockService.Setup(s => s.GetProductAsync(1)).ReturnsAsync(product);

        var controller = new ProductsController(mockService.Object);

        // Act
        var result = await controller.GetProduct(1);

        // Assert
        var okResult = Assert.IsType<OkObjectResult>(result.Result);
        var returnedProduct = Assert.IsType<ProductDto>(okResult.Value);
        Assert.Equal(1, returnedProduct.Id);
    }

    [Fact]
    public async Task GetProduct_NonExistingId_ReturnsNotFound()
    {
        // Arrange
        var mockService = new Mock<IProductService>();
        mockService.Setup(s => s.GetProductAsync(999)).ThrowsAsync(new KeyNotFoundException());

        var controller = new ProductsController(mockService.Object);

        // Act
        var result = await controller.GetProduct(999);

        // Assert
        Assert.IsType<NotFoundResult>(result.Result);
    }
}
```

Note that `ActionResult<T>` can be tested by inspecting the `Result` property (for error responses) or the `Value` property (for successful returns).

#### Testing Model Validation

If your controller uses model validation (which is automatic with `[ApiController]`), you can test validation by setting `ModelState` manually:

```csharp
[Fact]
public void CreateProduct_InvalidModel_ReturnsBadRequest()
{
    // Arrange
    var controller = new ProductsController(mockService.Object);
    controller.ModelState.AddModelError("Name", "Required");

    // Act
    var result = controller.CreateProduct(new ProductDto());

    // Assert
    Assert.IsType<BadRequestObjectResult>(result);
}
```

But this is less common because validation is typically tested separately or through integration tests.

#### Integration Testing Controllers

Integration tests give you the most confidence, as they exercise the full stack: routing, model binding, filters, authorization, and the action itself. Using `WebApplicationFactory`, you can send HTTP requests and assert on responses.

```csharp
[Fact]
public async Task PostProduct_ValidProduct_ReturnsCreated()
{
    // Arrange
    var newProduct = new CreateProductDto { Name = "New Product", Price = 19.99m };

    // Act
    var response = await _client.PostAsJsonAsync("/api/products", newProduct);

    // Assert
    Assert.Equal(HttpStatusCode.Created, response.StatusCode);
    var createdProduct = await response.Content.ReadFromJsonAsync<ProductDto>();
    Assert.NotNull(createdProduct);
    Assert.Equal("New Product", createdProduct.Name);
}
```

You can also verify that the product was actually added to the database by injecting the `DbContext` and querying it (though this couples the test to the database implementation).

#### Testing Authorization

If actions are protected with `[Authorize]`, you need to simulate an authenticated user. With integration tests, you can add an authentication handler that always succeeds.

```csharp
// In ConfigureWebHost of your custom factory
builder.ConfigureTestServices(services =>
{
    services.AddAuthentication(defaultScheme: "Test")
        .AddScheme<AuthenticationSchemeOptions, TestAuthHandler>("Test", options => { });
});

// Then create a client with a pre‑configured authentication header
_client = factory.CreateClient();
_client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Test");
```

Implement a simple `TestAuthHandler` that adds a default identity with claims.

### 16.5 Best Practices for Testing in ASP.NET Core

1. **Test behavior, not implementation** – Focus on what the code does, not how it does it. This makes tests less brittle.
2. **Keep tests independent** – Each test should set up its own data and not rely on others.
3. **Use meaningful names** – Method names should describe the scenario and expected outcome (e.g., `GetProduct_NonExistingId_ReturnsNotFound`).
4. **Prefer integration tests for critical paths** – They give more confidence but are slower. Use unit tests for complex logic.
5. **Use a clean database for integration tests** – Ensure each test runs with a known state. The in‑memory provider or a test container (e.g., Testcontainers) is great.
6. **Mock external services** – For dependencies like email, payment gateways, use mocks or fake implementations to avoid calling real external systems.
7. **Measure code coverage** – Tools like Coverlet can help identify untested code, but don’t obsess over 100% coverage.
8. **Run tests in CI** – Integrate your test suite into your continuous integration pipeline to catch failures early.

### 16.6 Testing with a Real Database (Testcontainers)

The in‑memory provider is fast but doesn’t behave exactly like a real database (e.g., it ignores constraints, has different transaction behavior). For more realistic integration tests, you can use **Testcontainers** to spin up a real database in a Docker container for your tests.

```bash
dotnet add package Testcontainers
```

Example with PostgreSQL:

```csharp
public class ProductsApiTests : IAsyncLifetime
{
    private readonly PostgreSqlContainer _postgres = new PostgreSqlBuilder()
        .WithImage("postgres:15")
        .WithDatabase("testdb")
        .WithUsername("test")
        .WithPassword("test")
        .Build();

    private WebApplicationFactory<Program> _factory;
    private HttpClient _client;

    public async Task InitializeAsync()
    {
        await _postgres.StartAsync();

        _factory = new WebApplicationFactory<Program>()
            .WithWebHostBuilder(builder =>
            {
                builder.ConfigureServices(services =>
                {
                    // Remove existing DbContext registration
                    var descriptor = services.SingleOrDefault(
                        d => d.ServiceType == typeof(DbContextOptions<AppDbContext>));
                    if (descriptor != null) services.Remove(descriptor);

                    // Add real PostgreSQL connection
                    services.AddDbContext<AppDbContext>(options =>
                        options.UseNpgsql(_postgres.GetConnectionString()));
                });
            });

        _client = _factory.CreateClient();
    }

    public async Task DisposeAsync() => await _postgres.DisposeAsync();

    // Tests here...
}
```

This gives you the highest fidelity, at the cost of slower test execution.

### Summary

You’ve now learned the essential testing techniques for ASP.NET Core applications:

- **Unit tests** with xUnit to verify isolated logic.
- **Mocking** with Moq to control dependencies and verify interactions.
- **Integration tests** with `WebApplicationFactory` to exercise the full HTTP stack.
- **Testing controllers** both in isolation and through integration.
- **Best practices** to keep your tests maintainable and reliable.

With a solid test suite, you can refactor with confidence, catch regressions early, and deliver higher‑quality software.

**Exercise:**

1. Add a new feature to your application (e.g., a simple calculation service) and write unit tests for it using xUnit.
2. Write a unit test for a controller action, mocking its dependencies with Moq.
3. Create an integration test for one of your API endpoints using `WebApplicationFactory`. If possible, use an in‑memory database.
4. (Optional) Set up Testcontainers to run a real database for your integration tests.

In the next chapter, **"Clean Architecture & Domain‑Driven Design,"** you’ll learn how to structure your application using proven architectural patterns that separate concerns, enhance testability, and keep your codebase maintainable as it grows.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='15. advanced_c_for_performance.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='17. clean_architecture_domain_driven_design.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
